In [1]:
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd

Load MNIST data, including rotated one

In [2]:
data = pd.read_csv("../data/Mnist/mnist_train.csv")
test_data = pd.read_csv("../data/Mnist/mnist_test.csv")
test_rotated_data = pd.read_csv("../data/Mnist/mnist_test_rotated.csv")

In [3]:
from numpy import float32
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import relu

# Define your CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.linear1 = nn.Linear(in_features = 784, out_features = 250)
        self.linear2 = nn.Linear(in_features = 250, out_features = 100)
        self.linear3 = nn.Linear(in_features = 100, out_features = 10)

    def forward(self, x):
        x = relu(self.linear1(x))
        x = relu(self.linear2(x))
        x = self.linear3(x)
        return x

# Define a custom dataset class
class ImageDFDataset(Dataset):
    def __init__(self, df:pd.DataFrame):
        self.images = df.drop(columns= 'label').to_numpy(float32)
        self.labels = df['label'].to_numpy()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx:int):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

In [4]:
# Convert the pandas DataFrame into a PyTorch dataset
dataset = ImageDFDataset(data)

# Create a dataloader for batching and shuffling the data
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create an instance of your CNN model
model = CNNClassifier()

# Define the loss function and optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = batch

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for monitoring the training progress
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 9.55907154083252
Epoch: 1, Loss: 5.795234203338623
Epoch: 1, Loss: 6.6799774169921875
Epoch: 1, Loss: 4.047008991241455
Epoch: 1, Loss: 3.6034369468688965
Epoch: 1, Loss: 3.0939221382141113
Epoch: 1, Loss: 3.0226449966430664
Epoch: 1, Loss: 2.5036890506744385
Epoch: 1, Loss: 1.9921165704727173
Epoch: 1, Loss: 1.2318768501281738
Epoch: 1, Loss: 2.9494550228118896
Epoch: 1, Loss: 2.045179843902588
Epoch: 1, Loss: 2.9020259380340576
Epoch: 1, Loss: 1.773103952407837
Epoch: 1, Loss: 0.4817787706851959
Epoch: 1, Loss: 1.6767425537109375
Epoch: 1, Loss: 0.8843610286712646
Epoch: 1, Loss: 1.031206488609314
Epoch: 1, Loss: 1.3144131898880005
Epoch: 1, Loss: 1.4904839992523193
Epoch: 1, Loss: 1.1639790534973145
Epoch: 1, Loss: 1.802611231803894
Epoch: 1, Loss: 1.523293375968933
Epoch: 1, Loss: 0.6133264899253845
Epoch: 1, Loss: 1.3828177452087402
Epoch: 1, Loss: 0.7136205434799194
Epoch: 1, Loss: 0.3862777352333069
Epoch: 1, Loss: 0.544114351272583
Epoch: 1, Loss: 0.727051258087

In [5]:
# Convert the pandas DataFrame into a PyTorch dataset
test_dataset = ImageDFDataset(test_data)

# Create a dataloader for batching and shuffling the data
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [6]:
# prepare to count predictions for each class
classes = tuple([x for x in range(0,10,1)])
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

Accuracy for class: 0 is 97.0 %
Accuracy for class: 1 is 99.1 %
Accuracy for class: 2 is 97.9 %
Accuracy for class: 3 is 96.1 %
Accuracy for class: 4 is 96.7 %
Accuracy for class: 5 is 96.6 %
Accuracy for class: 6 is 97.6 %
Accuracy for class: 7 is 98.1 %
Accuracy for class: 8 is 94.7 %
Accuracy for class: 9 is 95.2 %


In [7]:
# Convert the pandas DataFrame into a PyTorch dataset
test_rotated_dataset = ImageDFDataset(test_rotated_data)

# Create a dataloader for batching and shuffling the data
test_dataloader = DataLoader(test_rotated_dataset, batch_size=32, shuffle=True)

In [8]:
# prepare to count predictions for each class
classes = tuple([x for x in range(0,10,1)])
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

Accuracy for class: 0 is 43.1 %
Accuracy for class: 1 is 0.3 %
Accuracy for class: 2 is 8.8 %
Accuracy for class: 3 is 10.1 %
Accuracy for class: 4 is 12.6 %
Accuracy for class: 5 is 5.7 %
Accuracy for class: 6 is 35.8 %
Accuracy for class: 7 is 7.0 %
Accuracy for class: 8 is 22.4 %
Accuracy for class: 9 is 16.4 %


PicklingError: Can't pickle <class '__main__.CNNClassifier'>: it's not the same object as __main__.CNNClassifier

## Cifar

In [10]:
def unpickle(file:str):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def cifarToDf (path:str = "../data/cifar-10-batches-py\data_batch_1"):
    cifar = unpickle(path)
    cifar_df = pd.DataFrame(cifar[b'data'])
    cifar_df['label'] = cifar[b'labels']

    return cifar_df

In [14]:
data = cifarToDf()
data.head()

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
0,59,43,50,68,98,119,139,145,149,149,...,58,65,59,46,57,104,140,84,72,6
1,154,126,105,102,125,155,172,180,142,111,...,42,67,101,122,133,136,139,142,144,9
2,255,253,253,253,253,253,253,253,253,253,...,83,80,69,66,72,79,83,83,84,9
3,28,37,38,42,44,40,40,24,32,43,...,39,59,42,44,48,38,28,37,46,4
4,170,168,177,183,181,177,181,184,189,189,...,88,85,82,83,79,78,82,78,80,1


In [21]:
# Define your CNN model
from typing import Optional


class CNN_Cifar_Classifier(nn.Module):
    def __init__(self):
        super(CNN_Cifar_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(relu(self.conv1(x)))
        x = self.pool(relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define a custom dataset class
class ImageDFDataset(Dataset):
    def __init__(self, df:pd.DataFrame, TargetTransform : Optional [any]):
        super.__init__(TargetTransform = TargetTransform)
        self.images = df.drop(columns= 'label').to_numpy(float32)
        self.labels = df['label'].to_numpy()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx:int):
        if self.TargetTransform is not None:
            image = self.images[idx]
            label = self.labels[idx]
        else:
            image = self.images[idx]
            label = self.labels[idx]
        return image, label
    
    

In [24]:
import torchvision

In [32]:
import torchvision.transforms as transforms

tr = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=tr)

Files already downloaded and verified


In [33]:
# Convert the pandas DataFrame into a PyTorch dataset
dataset = ImageDFDataset(data)

# Create a dataloader for batching and shuffling the data
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Create an instance of your CNN model
model = CNN_Cifar_Classifier()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = batch

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for monitoring the training progress
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 3072]

In [ ]:
import torchvision
c = torchvision.datasets.CIFAR10

In [ ]:
c._check_integrity

<function torchvision.datasets.cifar.CIFAR10._check_integrity(self) -> bool>

In [1]:
from MNIST_CNN import *

In [4]:
local_data = True
path = '../data/Mnist'

In [6]:
pd.read_csv(f"../data/Mnist/mnist_train.csv")

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
rf"{path}/mnist_train.csv"

'../data/Mnist/mnist_train.csv'

In [10]:
f"../data/Mnist/mnist_train.csv"

'../data/Mnist/mnist_train.csv'

In [13]:
if local_data: # use locally stored mnist data 
    from utils import ImageDFDataset

    # train
    train_dataset = ImageDFDataset(f"{path}/mnist_train.csv", label_col_name='label')
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # test 
    test_dataset = ImageDFDataset(f"{path}/mnist_train.csv",label_col_name='label')
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

else: # Download and use Pytorch's Mnist data
    from torchvision.datasets.mnist import MNIST 
    train_dataloader = MNIST(root='./data', train=True, download=True)
    test_dataloader = MNIST(root='./data', train=False, download=False)

try:
    model = train(train_dataloader)
    if model is not None:
        save_model(model=model)
except Exception as e:
    print('Model saving unsuccessful')
    raise(e)

evaluate(test_dataloader, model)

Epoch: 1, Loss: 12.510440826416016
Epoch: 1, Loss: 11.904219627380371
Epoch: 1, Loss: 10.264046669006348
Epoch: 1, Loss: 8.737014770507812
Epoch: 1, Loss: 3.863938331604004
Epoch: 1, Loss: 3.7406437397003174
Epoch: 1, Loss: 6.478304862976074
Epoch: 1, Loss: 3.1666667461395264
Epoch: 1, Loss: 2.5292890071868896
Epoch: 1, Loss: 2.8387491703033447
Epoch: 1, Loss: 2.4554266929626465
Epoch: 1, Loss: 1.271627426147461
Epoch: 1, Loss: 2.3875977993011475
Epoch: 1, Loss: 2.7752115726470947
Epoch: 1, Loss: 2.4488754272460938
Epoch: 1, Loss: 1.1093847751617432
Epoch: 1, Loss: 1.411346435546875
Epoch: 1, Loss: 1.3269450664520264
Epoch: 1, Loss: 1.1993625164031982
Epoch: 1, Loss: 2.1020288467407227
Epoch: 1, Loss: 0.6989051103591919
Epoch: 1, Loss: 1.137413501739502
Epoch: 1, Loss: 0.772698163986206
Epoch: 1, Loss: 0.8411988615989685
Epoch: 1, Loss: 0.8809775710105896
Epoch: 1, Loss: 0.3578692674636841
Epoch: 1, Loss: 1.2404863834381104
Epoch: 1, Loss: 1.4472968578338623
Epoch: 1, Loss: 0.829821705

KeyboardInterrupt: 